In [1]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.layers import Embedding
import tensorflow as tf

In [4]:
df = pd.read_csv("../data/transformed/wrangled.csv")

In [5]:
df.head()

,Unnamed: 0,id,title,author,text,label,nb_names,exclamation,text_length
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,17,0,10
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,12,0,9
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,11,0,16
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,4,0,7
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,3,0,2


Regression

In [14]:
model = tf.keras.Sequential([
      tf.keras.layers.Dense(32, activation='relu', input_shape=(3,)),
      tf.keras.layers.Dense(32, activation='relu'),
      tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001),
                metrics=[tf.keras.metrics.Accuracy()])

In [15]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 32)                128       
_________________________________________________________________
dense_4 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 33        
Total params: 1,217
Trainable params: 1,217
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.fit(df[["nb_names", "exclamation", "text_length"]], df["label"], epochs=100, validation_split = 0.2)

Epoch 1/100
519/519 [==============================] - 1s 743us/step - loss: 0.3695 - accuracy: 0.0070 - val_loss: 0.3236 - val_accuracy: 0.0149
Epoch 2/100
519/519 [==============================] - 0s 569us/step - loss: 0.3278 - accuracy: 0.0287 - val_loss: 0.3236 - val_accuracy: 0.0308
Epoch 3/100
519/519 [==============================] - 0s 575us/step - loss: 0.3240 - accuracy: 0.0503 - val_loss: 0.3158 - val_accuracy: 0.0583
Epoch 4/100
519/519 [==============================] - 0s 591us/step - loss: 0.3229 - accuracy: 0.0585 - val_loss: 0.3156 - val_accuracy: 0.0681
Epoch 5/100
519/519 [==============================] - 0s 589us/step - loss: 0.3218 - accuracy: 0.0777 - val_loss: 0.3156 - val_accuracy: 0.0997
Epoch 6/100
519/519 [==============================] - 0s 568us/step - loss: 0.3209 - accuracy: 0.0865 - val_loss: 0.3151 - val_accuracy: 0.0975
Epoch 7/100
519/519 [==============================] - 0s 552us/step - loss: 0.3202 - accuracy: 0.0860 - val_loss: 0.3151 - val_ac

Tokenizer

In [19]:
# Shuffle data to improve performance
data = shuffle(df)

# Create list from data
text = []

for index, rows in df.iterrows(): 
    wordlist = str(rows["text"])
    text.append(wordlist)

label = list(df['label'])

# Create tokenizer
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(text)
sequences = tokenizer.texts_to_sequences(text)

# Rename for convienience
x_train = sequences
y_train = label

# Preprocess data
maxlen = 20
x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)

In [24]:
# Create tf model
model = Sequential()
model.add(Embedding(100000, 8, input_length=maxlen))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()

history = model.fit(np.array(x_train), np.array(y_train),
                    epochs=5,
                    batch_size=8,
                    validation_split=0.2)

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 20, 8)             800000    
_________________________________________________________________
flatten_5 (Flatten)          (None, 160)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 161       
Total params: 800,161
Trainable params: 800,161
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
2076/2076 [==============================] - 6s 3ms/step - loss: 0.4476 - acc: 0.7920 - val_loss: 0.3339 - val_acc: 0.8473
Epoch 2/5
2076/2076 [==============================] - 5s 3ms/step - loss: 0.2932 - acc: 0.8693 - val_loss: 0.3064 - val_acc: 0.8627
Epoch 3/5
2076/2076 [==============================] - 5s 2ms/step - loss: 0.2613 - acc: 0.8877 - val_loss: 0.30

Tokenizer + Regression